In [40]:
LANG = 'es' # or 'es'
TASK = 'gender' # or 'bot_human'

In [41]:
import os
from fastai.text import * 

In [42]:
torch.cuda.device_count()

1

In [43]:
proj_path = Path(os.getcwd()).parents[1]

In [44]:
data_path = proj_path/f'clean_data_csv_{LANG}'

In [45]:
df = pd.read_csv(data_path/f'all_{TASK}_data.csv')

In [46]:
df.head()

,gender,text
0,female,RT @ISosprisiones: @PP_Asturias @pablocasado_ ...
1,male,RT @ElNacionalWeb: Nos montan en camiones como...
2,female,RT @HistoriaEnFotos: Video de la semana: A vec...
3,female,RT @WinSportsTV: Colombia se ve favorita para ...
4,female,Os acordáis de cuando vuestra madre os chillab...


In [47]:
data_lm = TextLMDataBunch.from_csv(data_path, f'all_{TASK}_data.csv')

In [48]:
data_clas = TextClasDataBunch.from_csv(data_path, f'all_{TASK}_data.csv', 
                                       vocab=data_lm.train_ds.vocab, bs=32)

In [49]:
data_lm.save(f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas.save(f'data_clas_export_{TASK}_{LANG}.pkl')

In [50]:
data_lm = load_data(data_path, fname=f'data_lm_export_{TASK}_{LANG}.pkl')
data_clas = load_data(data_path, fname=f'data_clas_export_{TASK}_{LANG}.pkl', bs=16)

In [51]:
if LANG == 'es':
    weights_pretrained = 'model-30k-vocab-noqrnn'
    itos_pretrained = 'itos_pretrained'
    pretained_data = (weights_pretrained, itos_pretrained)
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5, pretrained_fnames=pretained_data)
else:
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,4.538115,4.214494,0.338568,01:41
2,4.219690,3.974395,0.352820,01:41
3,4.095427,3.921479,0.355145,01:41
4,3.960233,3.898384,0.356943,01:42
5,3.852141,3.878696,0.359102,01:41
6,3.752356,3.861223,0.361029,01:41
7,3.654377,3.845819,0.363146,01:41
8,3.531621,3.837612,0.363685,01:41
9,3.463402,3.835796,0.363971,01:41
10,3.400971,3.835129,0.364488,01:41


In [52]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
1,3.304552,3.822562,0.367824,01:53
2,3.257565,3.795461,0.371709,01:53
3,3.206107,3.775078,0.375215,01:53
4,3.119625,3.775993,0.376826,01:53
5,3.037339,3.783947,0.378051,01:53
6,2.937262,3.809418,0.377824,01:53
7,2.857004,3.825740,0.378831,01:53
8,2.776447,3.842632,0.378509,01:53
9,2.730201,3.853957,0.378375,01:53
10,2.697594,3.859846,0.378254,01:53


In [53]:
if LANG == 'es':
    test_string = "Nació en 1910"
else:
    test_string = "He was born in 1910"
learn.predict(test_string, n_words=50)

'Nació en 1910 y grillos regalaron Lima hasta barcos 😂 por lo pronto recibí a todos sus artistas . https : / / t.co / cuervo xxbos RT @juan_carlosesp : Tenemos un Pr . @transantiago desde Durmiendo . El semestre tiene demora , ya que ya'

In [54]:
learn.save_encoder(f'ft_enc_{TASK}_{LANG}')

In [55]:
learn_class = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder(f'ft_enc_{TASK}_{LANG}')

In [56]:
data_clas.show_batch()

text,target
xxbos 🐶 🐺 🐱 🐭 🐹 🐰 🐸 🐯 🐨 🐻 🐷 🐽 🐮 🐗 🐵 🐒 🐴 🐑 🐘 🐼 🐧 🐦 🐤 🐥 🐣 🐔 🐍 🐢 🐛 🐝 xxunk 🐞 xxunk 🐙 🐚 🐠 🐟 🐬 xxunk 🐋 xxunk 🐏 🐀 xxunk 🐅 🐇 xxunk 🐎 🐐 🐓 🐕 🐪 xxunk 🐂 xxunk xxunk xxunk 🐫 🐪 xxunk 🐈 🐩 🐾 💐 🌸 🌷 🍀 🌹 🌻,female
"xxbos ¡ xxup el xxup uribismo xxup juega xxup con xxup gambito xxup de xxup jep ! xxup ya xxup con xxup misuón xxup cumplida xxup de xxup curules xxup de xxup paz xxup hundidas a xxup costa xxup de xxup sacrificar xxup por xxup ahora xxup reforma xxup de xxup la xxup jep , xxup se xxup viene xxup la ' xxup constituyente ' xxmaj por falta de quórum",male
"xxbos @adnradiochile # lostenores xxmaj beñat porque con 5 lukas te armo 1 cena pa’ 20 . xxmaj llegaron como refuerzos jugadores reservas en sus clubes como xxmaj vilches y xxmaj bolados , con uno de los peores 3 que vi en xxmaj católica xxmaj voboril , con juveniles como xxmaj saavedra , xxmaj munder , xxmaj kuse y xxmaj rebolledo . xxmaj los motiva a todos jueguen o no",male
"xxbos # chilexlaclave ¿ xxmaj tanta mano había que meterle a la xxmaj roja @panchoeguiluz ? xxmaj rueda nomina a los amistosos en verso pero va a terminar en prosa en la xxmaj copa xxmaj américa con casi el mismo equipo de siempre , salvo xxmaj bravo lesión , xxmaj jara rendimiento y xxmaj vargas indisciplina . ( xxmaj guarden este 11 ) https : / / t.co / xxunk",male
"xxbos xxmaj la derecha es inepta hasta para planificar un golpe , la táctica con la que xxup fracasaron en xxmaj venezuela , durante 2017 , la xxup copian en xxmaj nicaragua . xxmaj veremos si les funciona . xxmaj por cierto , ves las fotos y comparas y te das cuenta que xxup nada de esa lucha es espontánea ... https : / / t.co / xxunk",male


In [57]:
learn_class.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,0.652774,0.677458,0.648745,01:59
2,0.659461,0.632329,0.639979,01:58
3,0.640644,0.628569,0.652645,02:02
4,0.659006,0.618700,0.658011,02:00
5,0.644479,0.613452,0.667611,01:59


In [58]:
learn.save(f'model_stage_1_{TASK}_{LANG}')